### Training and testing on original COVIDET dataset without including GPT-generated data

In [ ]:
!pip install bert-score

In [ ]:
!pip install datasets

In [ ]:
!pip install rouge_score

In [ ]:
!pip install psaw

In [5]:
from bert_score import score as bert_scr
import pandas as pd
from absl import app
from absl import flags

from datasets import load_metric

import json
import numpy as np
import os
import pandas as pd
import random
import torch
import torch.nn as nn

from datasets import load_dataset, Dataset
from transformers import AutoModel, AutoTokenizer, DataCollatorForSeq2Seq, BartForConditionalGeneration
from tqdm import tqdm

In [14]:
# To use the code, please first expand the Json files in the train_val_test directory by adding a Post key (along with the Reddit post text obtained using the PSAW wrapper) to each entry.

# Directory path containing JSON files
directory_path = "./data/train_val_test_anonymized-WITH_POSTS"

# Iterate through each JSON file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        file_path = os.path.join(directory_path, filename)

        # Read the existing JSON data from the file
        with open(file_path, "r") as file:
            data = json.load(file)

        # Iterate through each entry in the JSON data
        for key, entry in data.items():
            # Add the "Post" key with the Reddit post text
            post_text = entry.get("Reddit Post", "")
            entry["Post"] = post_text

        # Write the modified JSON data back to the file
        with open(file_path, "w") as file:
            json.dump(data, file, indent=2)

print("Post key added to each entry in JSON files.")

Post key added to each entry in JSON files.


In [15]:
import torch
torch.cuda.empty_cache()

In [20]:
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [ ]:
!TOKENIZERS_PARALLELISM=false python detection_summarization.py --emotion "disgust" --training_path /content/data/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json --validation_path /content/data/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json --test_path /content/data/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json --model facebook/bart-large-cnn --batch_size 1 --gradient_accumulation_steps 1 --results_detection_summarization "results" --learning_rate 0.01

2023-12-16 21:55:43.510353: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-16 21:55:43.510410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-16 21:55:43.511591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-16 21:55:44.744265: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/detection_summarization.py:43: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  

In [ ]:
!TOKENIZERS_PARALLELISM=false python emotion_summarization.py --emotion "disgust" --training_path /content/data/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json --validation_path /content/data/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json --test_path /content/data/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json --model facebook/bart-large-cnn --batch_size 1 --gradient_accumulation_steps 1 --results_detection_summarization "results" --learning_rate 0.01

### Train BART model on data that includes both COVIDET and GPT-generated data

In [ ]:
!TOKENIZERS_PARALLELISM=false python detection_summarization.py --emotion "disgust" --training_path /content/data/gpt_train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json --validation_path /content/data/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json --test_path /content/data/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json --model facebook/bart-large-cnn --batch_size 1 --gradient_accumulation_steps 1 --results_detection_summarization "results" --learning_rate 0.01